#

In [1]:
import os
import dotenv
from pathlib import Path

from langchain_core.messages import AIMessage, HumanMessage
from langchain_community.document_loaders.text import TextLoader
from langchain_community.document_loaders import (
    WebBaseLoader, 
    PyPDFLoader, 
    Docx2txtLoader,
)
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

dotenv.load_dotenv()

USER_AGENT environment variable not set, consider setting it to identify your requests.


True

In [2]:
def load_doc_to_db():
    # Use loader according to doc type
    if "rag_docs" in st.session_state and st.session_state.rag_docs:
        docs = [] 
        for doc_file in st.session_state.rag_docs:
            if doc_file.name not in st.session_state.rag_sources:
                if len(st.session_state.rag_sources) < DB_DOCS_LIMIT:
                    os.makedirs("source_files", exist_ok=True)
                    file_path = f"./source_files/{doc_file.name}"
                    with open(file_path, "wb") as file:
                        file.write(doc_file.read())

                    try:
                        if doc_file.type == "application/pdf":
                            loader = PyPDFLoader(file_path)
                        elif doc_file.name.endswith(".docx"):
                            loader = Docx2txtLoader(file_path)
                        elif doc_file.type in ["text/plain", "text/markdown"]:
                            loader = TextLoader(file_path)
                        #elif doc_file.name.endswith(".xlsx"):
                            #loader = UnstructuredExcelLoader(file_path, mode='elements')
                        elif doc_file.name.endswith(".xlsx"):
                            loader = AzureAIDocumentIntelligenceLoader(
                                 api_endpoint=os.getenv("AZ_OPENAI_ENDPOINT"),
                                 api_key=os.getenv("AZ_OPENAI_API_KEY"),
                                 file_path=file_path,
                                 api_model="prebuilt-layout"
                            )
                        elif doc_file.name.endswith(".py"):
                            loader = PythonLoader(file_path)
                        elif doc_file.name.endswith(".csv"):
                            loader = CSVLoader(file_path)
                        else:
                            st.warning(f"Document type {doc_file.type} not supported.")
                            continue
# Lets use a smarter lib https://python.langchain.com/v0.2/docs/integrations/document_loaders/microsoft_excel/ TBB 4/30/25
                        docs.extend(loader.load())
                        st.session_state.rag_sources.append(doc_file.name)

                    except Exception as e:
                        st.toast(f"Error loading document {doc_file.name}: {e}", icon="⚠️")
                        print(f"Error loading document {doc_file.name}: {e}")
                    
                    finally:
                        os.remove(file_path)

                else:
                    st.error(F"Maximum number of documents reached ({DB_DOCS_LIMIT}).")

        if docs:
            _split_and_load_docs(docs)
            st.toast(f"Document *{str([doc_file.name for doc_file in st.session_state.rag_docs])[1:-1]}* loaded successfully.", icon="✅")

In [3]:
# Load docs

doc_paths = [
    "docs/test_rag.pdf",
    "docs/test_rag.docx",
]

docs = [] 
for doc_file in doc_paths:
    file_path = Path(doc_file)

    try:
        if doc_file.endswith(".pdf"):
            loader = PyPDFLoader(file_path)
        elif doc_file.endswith(".docx"):
            loader = Docx2txtLoader(file_path)
        elif doc_file.endswith(".txt") or doc_file.name.endswith(".md"):
            loader = TextLoader(file_path)
        elif doc_file.name.endswith(".xlsx"):
            loader = AzureAIDocumentIntelligenceLoader(
                api_endpoint=os.getenv("AZ_OPENAI_ENDPOINT"),
                api_key=os.getenv("AZ_OPENAI_API_KEY"),
                file_path=file_path,
                api_model="prebuilt-layout"
                            )  
        else:
            print(f"Document type {doc_file.type} not supported.")
            continue

        docs.extend(loader.load())

    except Exception as e:
        print(f"Error loading document {doc_file.name}: {e}")


# Load URLs

url = "https://docs.streamlit.io/develop/quick-reference/release-notes"
try:
    loader = WebBaseLoader(url)
    docs.extend(loader.load())

except Exception as e:
    print(f"Error loading document from {url}: {e}")

In [4]:
docs

[Document(metadata={'producer': 'Microsoft® Word for Microsoft 365', 'creator': 'Microsoft® Word for Microsoft 365', 'creationdate': '2024-09-15T19:40:36+02:00', 'msip_label_1cf2ba15-c468-47c8-b178-cba8acf110ec_siteid': 'eb25818e-5bd5-49bf-99de-53e3e7b42630', 'msip_label_1cf2ba15-c468-47c8-b178-cba8acf110ec_method': 'Standard', 'msip_label_1cf2ba15-c468-47c8-b178-cba8acf110ec_enabled': 'True', 'author': 'Domingo Domènech Enric (ERNI)', 'moddate': '2024-09-15T19:40:36+02:00', 'source': 'docs/test_rag.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}, page_content='My favorite food is margarita pizza.  \nThere are 47588 bottles in the tr uck.'),
 Document(metadata={'source': 'docs/test_rag.docx'}, page_content='My favorite food is margarita pizza.\n\nThere are 47588 bottles in the truck.\n\nToday is April 30, 2025.'),
 Document(metadata={'source': 'https://docs.streamlit.io/develop/quick-reference/release-notes', 'title': 'Release notes - Streamlit Docs', 'description': 'A changelog 

In [5]:
# Split docs

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=5000,
    chunk_overlap=1000,
)

document_chunks = text_splitter.split_documents(docs)

In [6]:
# Tokenize and load the documents to the vector store

vector_db = Chroma.from_documents(
    documents=document_chunks,
    embedding=OpenAIEmbeddings(),
)

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [7]:
# Retrieve

def _get_context_retriever_chain(vector_db, llm):
    retriever = vector_db.as_retriever()
    prompt = ChatPromptTemplate.from_messages([
        MessagesPlaceholder(variable_name="messages"),
        ("user", "{input}"),
        ("user", "Given the above conversation, generate a search query to look up in order to get inforamtion relevant to the conversation, focusing on the most recent messages."),
    ])
    retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

    return retriever_chain

In [8]:
def get_conversational_rag_chain(llm):
    retriever_chain = _get_context_retriever_chain(vector_db, llm)

    prompt = ChatPromptTemplate.from_messages([
        ("system",
        """You are a helpful assistant. You will have to answer to user's queries.
        You will have some context to help with your answers, but now always would be completely related or helpful.
        You can also use your knowledge to assist answering the user's queries.\n
        {context}"""),
        MessagesPlaceholder(variable_name="messages"),
        ("user", "{input}"),
    ])
    stuff_documents_chain = create_stuff_documents_chain(llm, prompt)

    return create_retrieval_chain(retriever_chain, stuff_documents_chain)

In [15]:
# Augmented Generation

llm_stream_openai = ChatOpenAI(
    model="gpt-4o",  # Here you could use "o1-preview" or "o1-mini" if you already have access to them
    temperature=0.3,
    streaming=True,
)

llm_stream_anthropic = ChatAnthropic(
    model="claude-3-5-sonnet-20240620",
    temperature=0.3,
    streaming=True,
)

llm_stream = llm_stream_openai  # Select between OpenAI and Anthropic models for the response

messages = [
    {"role": "user", "content": "Hi"},
    {"role": "assistant", "content": "Hi there! How can I assist you today?"},
    {"role": "user", "content": "What is the latest version of Streamlit?"},
]
messages = [HumanMessage(content=m["content"]) if m["role"] == "user" else AIMessage(content=m["content"]) for m in messages]

conversation_rag_chain = get_conversational_rag_chain(llm_stream)
response_message = "*(RAG Response)*\n"
for chunk in conversation_rag_chain.pick("answer").stream({"messages": messages[:-1], "input": messages[-1].content}):
    response_message += chunk
    print(chunk, end="", flush=True)

messages.append({"role": "assistant", "content": response_message})

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [12]:
dotenv.load_dotenv()

True

In [11]:
load_dotenv()

NameError: name 'load_dotenv' is not defined

In [13]:
os.getenv("AZ_OPENAI_API_KEY")

'1oUiDhM7pfr7W5ssz6XadegvJQv6XS0TrAR0Ts7Az8vQvwPsFd2mJQQJ99BDACYeBjFXJ3w3AAABACOGYfUz'

In [14]:
import os
from openai import AzureOpenAI
from dotenv import load_dotenv
from langchain_openai import AzureChatOpenAI

load_dotenv()

llm_stream = AzureChatOpenAI(
    azure_endpoint=os.getenv("AZ_OPENAI_ENDPOINT"),
    openai_api_version="2024-12-01-preview",
    model_name="gpt-4o",
    openai_api_key=os.getenv("AZ_OPENAI_API_KEY"),
    openai_api_type="azure",
    temperature=0.3,
    streaming=True,
)



prompt = "Tell me something about Azure"

for chunk in llm_stream.stream(prompt):
    print(chunk.content, end="", flush=True)

Certainly! Microsoft Azure is a cloud computing platform and service offered by Microsoft. It provides a wide range of cloud services, including computing power, storage, networking, databases, artificial intelligence, machine learning, and more. Azure enables businesses and developers to build, deploy, and manage applications and services through Microsoft-managed data centers located around the globe.

Here are some key aspects of Azure:

### 1. **Cloud Service Models**
Azure supports various cloud service models to meet diverse needs:
   - **Infrastructure as a Service (IaaS):** Provides virtualized computing resources like virtual machines (VMs), storage, and networking.
   - **Platform as a Service (PaaS):** Offers tools and frameworks for developers to build, test, and deploy applications without worrying about underlying infrastructure.
   - **Software as a Service (SaaS):** Hosts applications that users can access over the internet, such as Office 365.

### 2. **Key Features**
